In [1]:
import uproot
import numpy as np

from awkward.array.jagged import JaggedArray

In [2]:
events_path = "../mg5_data/SM-process_spin-ON_100k/Events/run_01_decayed_1/tag_1_delphes_events.root"
events = uproot.open(events_path)["Delphes"]

## Jet Cuts

In [3]:
def select_jet(events) -> JaggedArray:
    """Create boolean mask to apply jet selection criteria from ATLAS

    :param events: Delphes event TTree containing
    :type events: TTree
    :return: boolean mask to select jets in events
    :rtype: JaggedArray
    """
    jet_pt = events["Jet.PT"].array()
    jet_phi = events["Jet.Phi"].array()
    jet_eta = events["Jet.Eta"].array()
    electron_phi = events["Electron.Phi"].array()
    electron_eta = events["Electron.Eta"].array()
    muon_phi = events["Muon.Phi"].array()
    muon_eta = events["Muon.Eta"].array()

    pt_mask = jet_pt > 25
    eta_mask = np.abs(jet_eta) < 2.5
    electron_dR_mask = []
    muon_dR_mask = []
    for event_idx in range(len(events)):
        jet_phi_idx = jet_phi[event_idx]
        jet_eta_idx = jet_eta[event_idx]
        electron_dR_event_mask = np.ones_like(jet_phi_idx, dtype=int)
        muon_dR_event_mask = np.ones_like(jet_phi_idx, dtype=int)
        for elec_idx in range(len(electron_phi[event_idx])):
            dPhi = jet_phi_idx - electron_phi[event_idx][elec_idx]
            dEta = jet_eta_idx - electron_eta[event_idx][elec_idx]
            dR = np.sqrt(dPhi**2 + dEta**2)
            electron_dR_event_mask *= (dR > 0.2)
        for muon_idx in range(len(muon_phi[event_idx])):
            dPhi = jet_phi_idx - muon_phi[event_idx][muon_idx]
            dEta = jet_eta_idx - muon_eta[event_idx][muon_idx]
            dR = np.sqrt(dPhi**2 + dEta**2)
            muon_dR_event_mask *= (dR > 0.4)
        electron_dR_mask.append(electron_dR_event_mask.astype(bool))
        muon_dR_mask.append(muon_dR_event_mask.astype(bool))
    electron_dR_mask = JaggedArray.fromiter(electron_dR_mask)
    muon_dR_mask = JaggedArray.fromiter(muon_dR_mask)
    return pt_mask, eta_mask, electron_dR_mask, muon_dR_mask

In [4]:
jet_pt_mask, jet_eta_mask, jet_electron_dR_mask, jet_muon_dR_mask = select_jet(events)

full_jets_mask = jet_pt_mask * jet_eta_mask * jet_electron_dR_mask * jet_muon_dR_mask
bjets_mask = events["Jet.BTag"].array()[full_jets_mask].astype(bool)

## Electron

In [5]:
def select_electron(events) -> JaggedArray:
    """Create boolean mask to apply electron selection criteria from ATLAS

    :param events: Delphes event TTree containing
    :type events: TTree
    :return: boolean mask to select electrons in events
    :rtype: JaggedArray
    """
    jet_phi = events["Jet.Phi"].array()
    jet_eta = events["Jet.Eta"].array()
    jet_mass = events["Jet.Mass"].array()
    jet_pt = events["Jet.PT"].array()
    electron_pt = events["Electron.PT"].array()
    electron_phi = events["Electron.Phi"].array()
    electron_eta = events["Electron.Eta"].array()

    pt_mask = electron_pt > 25
    eta_mask1 = (np.abs(electron_eta) < 2.5) * (np.abs(electron_eta) > 1.52)
    eta_mask2 = np.abs(electron_eta) < 1.37
    eta_mask = eta_mask1 + eta_mask2
    jet_dR_mask = []
    for event_idx in range(len(events)):
        electron_phi_idx = electron_phi[event_idx]
        electron_eta_idx = electron_eta[event_idx]
        jet_dR_event_mask = np.ones_like(electron_phi_idx, dtype=int)
        for jet_idx in range(len(jet_phi[event_idx])):
            jet_eta_idx = jet_eta[event_idx][jet_idx]
            jet_mass_idx = jet_mass[event_idx][jet_idx]
            jet_pt_idx = jet_pt[event_idx][jet_idx]
            jet_rapidity = jet_eta_idx - (np.tanh(jet_eta_idx)/2)*(jet_mass_idx/jet_pt_idx)**2
            dPhi = electron_phi_idx - jet_phi[event_idx][jet_idx]
            dEta = electron_eta_idx - jet_rapidity
            dR = np.sqrt(dPhi**2 + dEta**2)
            jet_dR_event_mask *= (dR > 0.4)
        jet_dR_mask.append(jet_dR_event_mask.astype(bool))
    jet_dR_mask = JaggedArray.fromiter(jet_dR_mask)
    return pt_mask, eta_mask, jet_dR_mask

In [6]:
electron_pt_mask, electron_eta_mask, electron_jet_dR_mask = select_electron(events)
full_electron_mask = electron_pt_mask * electron_eta_mask * electron_jet_dR_mask

## Muon

In [7]:
def select_muon(events) -> JaggedArray:
    """Create boolean mask to apply muon selection criteria from ATLAS

    :param events: Delphes event TTree containing
    :type events: TTree
    :return: boolean mask to select muon in events
    :rtype: JaggedArray
    """
    jet_phi = events["Jet.Phi"].array()
    jet_eta = events["Jet.Eta"].array()
    muon_pt = events["Muon.PT"].array()
    muon_phi = events["Muon.Phi"].array()
    muon_eta = events["Muon.Eta"].array()

    pt_mask = muon_pt > 25
    eta_mask = np.abs(muon_eta) < 2.5
    jet_dR_mask = []
    for event_idx in range(len(events)):
        muon_phi_idx = muon_phi[event_idx]
        muon_eta_idx = muon_eta[event_idx]
        jet_dR_event_mask = np.ones_like(muon_phi_idx, dtype=int)
        for jet_idx in range(len(jet_phi[event_idx])):
            dPhi = muon_phi_idx - jet_phi[event_idx][jet_idx]
            dEta = muon_eta_idx - jet_eta[event_idx][jet_idx]
            dR = np.sqrt(dPhi**2 + dEta**2)
            jet_dR_event_mask *= (dR > 0.4)
        jet_dR_mask.append(jet_dR_event_mask.astype(bool))
    jet_dR_mask = JaggedArray.fromiter(jet_dR_mask)
    return pt_mask, eta_mask, jet_dR_mask

In [8]:
muon_pt_mask, muon_eta_mask, muon_jet_dR_mask = select_muon(events)
full_muon_mask = muon_pt_mask * muon_eta_mask * muon_jet_dR_mask

## Cutflow

### Single-Particle Cuts

In [9]:
selected_events = 0
for event in full_electron_mask:
    if 1 <= np.sum(event) <= 2:
        selected_events += 1
print(f"Events after electron cuts: {selected_events:,}")

Events after electron cuts: 37,382


In [10]:
selected_events = 0
for event in full_muon_mask:
    if 1 <= np.sum(event) <= 2:
        selected_events += 1
print(f"Events after muon cuts: {selected_events:,}")

Events after muon cuts: 49,490


In [11]:
selected_events = 0
for event in bjets_mask:
    if np.sum(event) >= 2:
        selected_events += 1
print(f"Events after b-jet cuts: {selected_events:,}")

Events after b-jet cuts: 31,950


### Two-Particle Cuts

In [12]:
selected_events = 0
for e_event, mu_event in zip(full_electron_mask, full_muon_mask):
    if np.sum(e_event) + np.sum(mu_event) == 2:
        selected_events += 1
print(f"Events after electron and muon cuts: {selected_events:,}")

Events after electron and muon cuts: 24,896


In [13]:
selected_events = 0
for e_event, b_event in zip(full_electron_mask, bjets_mask):
    if (1 <= np.sum(e_event) <= 2) and (np.sum(b_event) >= 2):
        selected_events += 1
print(f"Events after electron and b-jet cuts: {selected_events:,}")

Events after electron and b-jet cuts: 11,707


In [14]:
selected_events = 0
for mu_event, b_event in zip(full_muon_mask, bjets_mask):
    if (1 <= np.sum(mu_event) <= 2) and (np.sum(b_event) >= 2):
        selected_events += 1
print(f"Events after muon and b-jet cuts: {selected_events:,}")

Events after muon and b-jet cuts: 15,587


### Full Cuts

In [15]:
selected_events = 0
for e_event, mu_event, b_event in zip(full_electron_mask, full_muon_mask, bjets_mask):
    if (np.sum(e_event) + np.sum(mu_event) == 2) and (np.sum(b_event) >= 2):
        selected_events += 1
print(f"Events after full cuts: {selected_events:,}")

Events after full cuts: 7,663
